In [1]:
import pandas as pd

In [4]:
df = pd.read_csv("spotify_millsongdata.csv.csv.csv")

In [5]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [6]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [7]:
df.shape

(57650, 4)

In [8]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [9]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [10]:
df.head(10)

,artist,song,text
0,Lucky Dube,Prisoner,Somebody told me about it \r\nWhen I was stil...
1,Dean Martin,Beau James,He walked along Broadway \r\nThey called him ...
2,Miley Cyrus,The Floyd Song (Sunrise),[Chorus] \r\nThe sunrise insists on gladness ...
3,Genesis,Uncertain Weather,A day of uncertain weather \r\nCaptured in a ...
4,Alabama,Fireworks,There are people in this country who work hard...
5,Keith Green,Song For Josiah,"Oh my son, you were born in a world that hates..."
6,Kenny Loggins,St. Judy's Comet,Little sleepy boy \r\nDo you know what time i...
7,Freestyle,These City Streets,"""these city streets got a nigga packin heat fo..."
8,ZZ Top,Planet Of Women,"What can I do, I'm a nervous wreck? \r\nThere..."
9,Whitney Houston,Salute,Eh eh eh eh eh eh oh \r\n \r\nSay you want t...


In [11]:
df['text'][0]

"Somebody told me about it  \r\nWhen I was still a little boy  \r\nHe said to me, crime does not pay  \r\nHe said to me, education is the key, yeah  \r\nAs a little boy I thought I know  \r\nWhat I was doing, yeah man  \r\nBut today here I am in jail  \r\n  \r\nI' m a prisoner  \r\nI' m a prisoner  \r\nI' m a prisoner  \r\n  \r\nI looked all around me  \r\nBut to see nothing  \r\nbut four grey walls staring at me  \r\nthe policeman said to me, son  \r\nThey won' t build no schools anymore  \r\nAll they' ll build will be prison, prison  \r\nAll they' ll build will be prison, prison  \r\nAll they' ll build will be prison, prison  \r\n'Cause today, yeah  \r\n  \r\nI am a prisoner  \r\nI am a prisoner  \r\nI' m a prisoner  \r\nDear lord  \r\n  \r\nI asked the policeman and said  \r\nHow much must I pay for my freedom?  \r\nHe said to me, son  \r\nThey won't build no schools anymore  \r\nThey won't build no hospitals  \r\nThey won't build no hospitals  \r\nAll they' ll build will be prison,

In [12]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [13]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [14]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [15]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [18]:
similarity[0]

array([1.00000000e+00, 1.50656330e-03, 0.00000000e+00, ...,
       2.94560912e-04, 6.35876518e-03, 1.29604611e-02])

In [19]:
df[df['song'] == 'Salute']

,artist,song,text
9,Whitney Houston,Salute,eh eh eh eh eh eh oh say you want to walk away...


In [20]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [21]:
recommendation('Salute')

['Telephone',
 'Alphadog',
 'Along Came Jones',
 'Hammer To Fall',
 'Try',
 'Soldier On',
 "Just When I'd Stopped Lovin' You",
 "Nobody's Soldier",
 'Lets Talk Dirty In Hawaiian',
 'When A Soldier Makes It Home',
 'Campfire',
 'Mr McGee',
 'Hollow',
 'I Call It Love',
 "Feelin' So Good",
 'Good Enough For Me',
 'I Would Die For You',
 'My Fun House',
 'Walking In Circles',
 'This Must Be Love']

In [22]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))